# Receipt Data Extraction: Using AI_EXTRACT for Automatic Structure Detection

## Overview
This notebook demonstrates how to use Snowflake's AI_EXTRACT function to automatically extract structured information from ad-campaign receipt PDFs. Unlike AI_COMPLETE, AI_EXTRACT automatically detects and extracts structured data without requiring detailed schemas.

## What We'll Accomplish
- Parse PDF receipts from the `@RECEIPTS_PROCESSING_DB.RAW.RECEIPTS` stage using `AI_PARSE_DOCUMENT`
- Extract structured receipt data automatically using `AI_EXTRACT`
- Transform unstructured receipt content into structured, analyzable data with minimal configuration

## Prerequisites
- Access to Snowflake with Cortex AI features enabled
- Receipts uploaded to `@RECEIPTS_PROCESSING_DB.RAW.RECEIPTS` stage
- RECEIPTS_PROCESSING_DB database and RAW schema configured
- Appropriate permissions for the ETL service role


In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# Use Snowpark for our analyses
from snowflake.snowpark.context import get_active_session
session = get_active_session()


## Step 1: Environment Setup and Session Initialization

Setting up our environment by importing necessary packages and establishing a Snowflake session.

### Key Components:
- **Streamlit**: For building interactive applications
- **Pandas**: For data manipulation and analysis
- **Snowpark Session**: Connection to Snowflake and access to Cortex AI capabilities

The `get_active_session()` retrieves our Snowflake session for executing SQL and using AI features.

## Step 2: Set Database and Schema Context
Setting the working context for our session:

- **Database**: `RECEIPTS_PROCESSING_DB` - Our receipt processing database
- **Schema**: `RAW` - The schema containing our receipts stage

This ensures all operations execute within the correct context without needing to fully qualify object names.


In [ ]:
session.sql("USE ROLE SYSADMIN").collect()

# Set warehouse for AI processing
session.sql("USE WAREHOUSE RECEIPTS_PARSE_COMPLETE_WH").collect()

session.sql("ALTER WAREHOUSE RECEIPTS_PARSE_COMPLETE_WH SET WAREHOUSE_SIZE='XSMALL'").collect()

session.use_database('RECEIPTS_PROCESSING_DB')
session.use_schema('RAW')


## Step 3: Import Processing Libraries

Importing libraries for document AI processing:

### Key Imports:
- **JSON**: For handling structured data extracted by AI_EXTRACT

Note: AI_EXTRACT requires minimal configuration compared to AI_COMPLETE, so we don't need Pydantic schemas or complex prompt management.


In [ ]:
import json


## Step 4: Explore Available Receipts

Before processing, let's see what receipt files are available in our stage.

### What This Shows:
- File names and paths of receipts ready for processing
- File sizes and metadata
- Upload timestamps

The `DIRECTORY()` function provides a view of all files in the `@RECEIPTS_PROCESSING_DB.RAW.RECEIPTS` stage, essential for understanding our data source.


In [ ]:
--REMOVE @RECEIPTS_PROCESSING_DB.RAW.RECEIPTS; -- REMOVES ALL FILES FROM THE STAGE
ALTER STAGE RECEIPTS_PROCESSING_DB.RAW.RECEIPTS REFRESH;
SELECT * FROM DIRECTORY(@RECEIPTS_PROCESSING_DB.RAW.RECEIPTS);


## Step 7: Define Extraction Fields

With AI_EXTRACT, we simply list the fields we want to extract. The AI automatically:
- Identifies these fields in the document
- Determines appropriate data types
- Extracts values without needing detailed schemas or prompts

This is much simpler than AI_COMPLETE which requires detailed JSON schemas and prompts.


In [ ]:
# Define what fields we want AI_EXTRACT to find
# AI_EXTRACT automatically identifies and extracts these without needing detailed schemas
extraction_fields = [
    'vendor_name',
    'receipt_id',
    'transaction_date',
    'payment_method',
    'customer_name',
    'company_name',
    'campaign_name',
    'period_startdate',
    'period_enddate',
    'content_types',
    'ad_formats',
    'subtotal',
    'tax',
    'total',
    'cpm',
    'ctr',
    'bounce_rate',
    'pricing_model',
    'daily_budget',
    'total_budget',
    'geography',
    'demographics',
    'age_range',
    'devices'
]

print(f"Will extract {len(extraction_fields)} fields using AI_EXTRACT")


In [ ]:
resp_schema = """
{
    'type': 'json',
    'schema': {
        'type': 'object',
        'properties': {
            'vendor': {
                'type': 'object',
                'properties': {
                    'vendor_name': {'type': 'string'}
                },
                'required': ['vendor_name']
            },
            'transaction': {
                'type': 'object',
                'properties': {
                    'receipt_id': {'type': 'string'},
                    'date': {'type': 'string'},
                    'payment_method': {'type': 'string'}
                },
                'required': ['receipt_id', 'date', 'payment_method']
            },
            'customer': {
                'type': 'object',
                'properties': {
                    'customer_name': {'type': 'string'},
                    'company_name': {'type': 'string'}
                },
                'required': ['company_name']
            },
            'campaign': {
                'type': 'object',
                'properties': {
                    'name': {'type': 'string'},
                    'content_types': {'type': 'string'},
                    'ad_formats': {'type': 'array'},
                    'period_startdate': {'type': 'string'},
                    'period_enddate': {'type': 'string'},
                    'budget': {'type':'number'}
                },
                'required': ['name', 'content_types', 'ad_formats', 'period_startdate', 'period_enddate']
            },
            'financials': {
                'type': 'object',
                'properties': {
                    'line_items': {'type': 'object'},
                    'subtotal': {'type': 'number'},
                    'tax': {'type': 'number'},
                    'total': {'type': 'number'}
                },
                'required': ['total', 'line_items', 'subtotal', 'tax']
            },
            'metrics': {
                'type': 'object',
                'properties': {
                    'cpm': {'type': 'string', 'description':'Cost Per Milli, abbreviated as CPM'},
                    'ctr': {'type': 'string', 'description':'Click-through rate, abbreviated as CTR'},
                    'bounce_rate': {'type': 'string', 'description':'Bounce Rate, sometimes just referred to as Bounce, a % value'},
                    'targets': {'type': 'object'},
                    'pricing_model': {'type': 'string'}
                },
                'required': ['cpm', 'ctr', 'bounce_rate', 'targets', 'pricing_model']
            },
            'budget':{
                'type': 'object',
                'properties': {
                    'daily_budget': {'type': 'string'},
                    'total_budget': {'type': 'string'}
                },
                'required': ['daily_budget', 'total_budget']
            },
            'targeting': {
                'type': 'object',
                'properties': {
                    'geography': {'type': 'array'},
                    'demographics': {'type': 'string'},
                    'age_range': {'type': 'string'},
                    'devices': {'type': 'string'}
                },
                'required': ['geography', 'demographics', 'age_range', 'devices']
            }
        },
        'required': ['vendor', 'transaction', 'customer', 'campaign', 'financials', 'metrics', 'budget', 'targeting']
    }
}
"""


## Step 8: Extract Structured Data with AI_EXTRACT (Incremental)

Using Snowflake's AI_EXTRACT to automatically extract structured receipt data - only processing newly parsed receipts.

### Incremental Extraction:
- **CREATE TABLE IF NOT EXISTS**: Preserves existing extracted data
- **INSERT INTO**: Adds only new extractions
- **WHERE NOT IN**: Only processes receipts not already in extracted_receipt_data_via_ai_extract
- **Saves Costs**: Avoids re-running expensive AI_EXTRACT on same receipts

### AI_EXTRACT vs AI_COMPLETE:
- ✅ **Simpler**: No need for complex prompts or JSON schemas
- ✅ **Automatic**: AI determines data types and extraction logic
- ✅ **Flexible**: Just provide field names, AI handles the rest

The AI reads each new receipt and automatically extracts vendor details, transaction info, campaign details, financial totals, performance metrics, and targeting parameters into structured data.


In [ ]:
# Create extracted_receipt_data_via_ai_extract table if it doesn't exist
session.sql("""
CREATE TABLE IF NOT EXISTS extracted_receipt_data_via_ai_extract (
    relative_path STRING,
    content STRING,
    extracted_data VARIANT
)
""").collect()

# Build field list for AI_EXTRACT
fields_str = ', '.join([f"'{field}'" for field in extraction_fields])

# Only extract data from newly parsed receipts
query = f"""
INSERT INTO extracted_receipt_data_via_ai_extract
SELECT
    relative_path,
    content,
    AI_EXTRACT(
        content,
        ARRAY_CONSTRUCT({fields_str})
    ) as extracted_data
FROM parsed_receipts
WHERE relative_path NOT IN (SELECT relative_path FROM extracted_receipt_data_via_ai_extract)
"""

print("Query prepared for AI_EXTRACT processing")


In [ ]:
result = session.sql(query).collect()
# Get the actual number of rows inserted from the result metadata
rows_inserted = result[0]['number of rows inserted'] if result else 0
print(f"✓ Extracted data from {rows_inserted} new receipt(s) using AI_EXTRACT")


In [ ]:
# View the extracted data
result_df = session.table('extracted_receipt_data_via_ai_extract').to_pandas()
result_df.head()


## Step 9: Preview Extracted Receipt Data

Each row contains a complete structured representation of a receipt with all extracted fields in VARIANT format, ready for flattening and analysis.


In [ ]:
# Parse and flatten the extracted data for analysis
flattened_df = session.sql("""
SELECT
    relative_path,
    extracted_data:vendor_name::STRING AS vendor_name,
    extracted_data:receipt_id::STRING AS receipt_id,
    TRY_TO_DATE(extracted_data:transaction_date::STRING) AS transaction_date,
    extracted_data:payment_method::STRING AS payment_method,
    extracted_data:company_name::STRING AS company_name,
    extracted_data:campaign_name::STRING AS campaign_name,
    TRY_TO_DATE(extracted_data:period_startdate::STRING) AS period_startdate,
    TRY_TO_DATE(extracted_data:period_enddate::STRING) AS period_enddate,
    extracted_data:content_types::STRING AS content_types,
    TRY_TO_DECIMAL(REPLACE(REPLACE(extracted_data:total::STRING, '$', ''), ',', ''), 10, 2) AS total_amount,
    TRY_TO_DECIMAL(REPLACE(extracted_data:cpm::STRING, '$', ''), 10, 2) AS cpm,
    TRY_TO_DECIMAL(REPLACE(extracted_data:ctr::STRING, '%', ''), 10, 2) AS ctr,
    TRY_TO_DECIMAL(REPLACE(extracted_data:bounce_rate::STRING, '%', ''), 10, 2) AS bounce_rate,
    extracted_data:pricing_model::STRING AS pricing_model,
    TRY_TO_DECIMAL(REPLACE(REPLACE(extracted_data:daily_budget::STRING, '$', ''), ',', ''), 10, 2) AS daily_budget,
    TRY_TO_DECIMAL(REPLACE(REPLACE(extracted_data:total_budget::STRING, '$', ''), ',', ''), 10, 2) AS total_budget
FROM extracted_receipt_data_via_ai_extract
""").to_pandas()

flattened_df.head(10)


## Step 10: Flatten and Query Extracted Data

Converting nested VARIANT data into a flat table showing key receipt fields including vendor, transaction date, campaign details, total amount, and performance metrics.


In [ ]:
# Create a view with flattened receipt data
# Strip non-numeric characters ($, %, commas) before converting to numbers
session.sql("""
CREATE OR REPLACE VIEW receipt_analytics_ai_extract_vw AS
SELECT
    relative_path AS receipt_filename,
    extracted_data:vendor_name::STRING AS vendor_name,
    extracted_data:receipt_id::STRING AS receipt_id,
    TRY_TO_DATE(extracted_data:transaction_date::STRING) AS transaction_date,
    extracted_data:payment_method::STRING AS payment_method,
    extracted_data:company_name::STRING AS company_name,
    extracted_data:customer_name::STRING AS customer_name,
    extracted_data:campaign_name::STRING AS campaign_name,
    TRY_TO_DATE(extracted_data:period_startdate::STRING) AS period_startdate,
    TRY_TO_DATE(extracted_data:period_enddate::STRING) AS period_enddate,
    extracted_data:content_types::STRING AS content_types,
    -- Strip $ and , from financial values and preserve decimals
    TRY_TO_DECIMAL(REPLACE(REPLACE(extracted_data:subtotal::STRING, '$', ''), ',', ''), 10, 2) AS subtotal,
    TRY_TO_DECIMAL(REPLACE(REPLACE(extracted_data:tax::STRING, '$', ''), ',', ''), 10, 2) AS tax,
    TRY_TO_DECIMAL(REPLACE(REPLACE(extracted_data:total::STRING, '$', ''), ',', ''), 10, 2) AS total_amount,
    -- Strip $ from CPM and preserve decimals
    TRY_TO_DECIMAL(REPLACE(extracted_data:cpm::STRING, '$', ''), 10, 2) AS cpm,
    -- Strip % from CTR and Bounce Rate and preserve decimals
    TRY_TO_DECIMAL(REPLACE(extracted_data:ctr::STRING, '%', ''), 10, 2) AS ctr_percent,
    TRY_TO_DECIMAL(REPLACE(extracted_data:bounce_rate::STRING, '%', ''), 10, 2) AS bounce_rate_percent,
    extracted_data:pricing_model::STRING AS pricing_model,
    -- Strip $ and , from budget values and preserve decimals
    TRY_TO_DECIMAL(REPLACE(REPLACE(extracted_data:daily_budget::STRING, '$', ''), ',', ''), 10, 2) AS daily_budget,
    TRY_TO_DECIMAL(REPLACE(REPLACE(extracted_data:total_budget::STRING, '$', ''), ',', ''), 10, 2) AS campaign_budget,
    extracted_data:age_range::STRING AS age_range,
    CURRENT_TIMESTAMP() AS processed_at
FROM extracted_receipt_data_via_ai_extract
""").collect()

print("✓ receipt_analytics_ai_extract_vw created successfully!")


In [ ]:
SELECT * FROM receipt_analytics_ai_extract_vw LIMIT 10;


## Step 11: Create Analytics View

Creating a view with flattened receipt data for analytics with proper type conversion, descriptive column names, and processing timestamp. Ready for dashboards and reporting!


In [ ]:
# Example analytics: Spending by vendor
session.sql("""
SELECT 
    vendor_name,
    COUNT(*) AS receipt_count,
    SUM(total_amount) AS total_spending,
    AVG(total_amount) AS avg_receipt_amount,
    AVG(cpm) AS avg_cpm,
    AVG(ctr_percent) AS avg_ctr
FROM receipt_analytics_ai_extract_vw
GROUP BY vendor_name
ORDER BY total_spending DESC
""").to_pandas()


## Step 12: Analyze Spending by Vendor

Analyzing receipt count, total spending, average amounts, and performance metrics (CPM, CTR) by vendor to identify top advertising partners and their performance.


In [ ]:
# Campaign type analysis
session.sql("""
SELECT 
    content_types,
    COUNT(*) AS campaign_count,
    AVG(total_amount) AS avg_spending,
    AVG(cpm) AS avg_cpm,
    AVG(ctr_percent) AS avg_ctr,
    AVG(bounce_rate_percent) AS avg_bounce_rate
FROM receipt_analytics_ai_extract_vw
WHERE content_types IS NOT NULL
GROUP BY content_types
ORDER BY campaign_count DESC
""").to_pandas()


## Step 13: Analyze by Campaign Content Type

Comparing performance between Display, Video, and mixed campaigns to optimize content strategy and budget allocation.


In [ ]:
# Performance metrics by pricing model
session.sql("""
SELECT 
    pricing_model,
    COUNT(*) AS receipt_count,
    AVG(cpm) AS avg_cpm,
    AVG(ctr_percent) AS avg_ctr,
    AVG(bounce_rate_percent) AS avg_bounce_rate,
    SUM(total_amount) AS total_spending
FROM receipt_analytics_ai_extract_vw
WHERE pricing_model IS NOT NULL
GROUP BY pricing_model
ORDER BY receipt_count DESC
""").to_pandas()


## Step 14: Analyze by Pricing Model

Understanding performance across different pricing models (CPM, CPC, CPA, CPV, Flat Rate) to determine which delivers the best ROI.


## Summary

### What We've Accomplished:

1. ✅ **Parsed Receipts**: Extracted text from PDF receipts using AI_PARSE_DOCUMENT
2. ✅ **Automatic Extraction**: Used AI_EXTRACT to automatically identify and extract structured data
3. ✅ **Analytics View**: Created `receipt_analytics_ai_extract_vw` with flattened, queryable data
4. ✅ **Generated Insights**: Analyzed spending, performance, and campaign metrics

### Key Metrics Captured:
- **Financial**: Subtotal, tax, total amounts
- **Performance**: CPM, CTR, Bounce Rate
- **Campaign**: Display formats, video placements, targeting
- **Budget**: Daily and total campaign budgets

### AI_EXTRACT Advantages:
- ✅ **Simpler Setup**: No complex schemas or prompts needed
- ✅ **Automatic Detection**: AI identifies field types and structures
- ✅ **Flexible**: Just specify field names, AI handles extraction logic
- ✅ **Faster Development**: Less configuration means faster implementation

### Tables & Views Created:
1. `parsed_receipts` - Raw parsed text from PDFs (shared with AI_COMPLETE approach)
2. `extracted_receipt_data_via_ai_extract` - Structured data extracted with AI_EXTRACT
3. `receipt_analytics_ai_extract_vw` - Flattened, analytics-ready view

---

**Your receipt data is now structured with AI_EXTRACT, queryable, and ready for analysis!** 📊
